In [13]:
import warnings
import pandas as pd
from datetime import datetime
from datetime import date
import sqlalchemy as sql
from pathlib import Path
warnings.filterwarnings('ignore')  # Hide warnings
#Library - Project3 
import CryptoDownloadData as coinData
import CryptoPerfSummary as coinAnalytic

#DataBase Tables
# Database connection string
crypto_data_connection_string = 'sqlite:///./Reference/crypto.db'
# Database engine
crypto_data_engine = sql.create_engine(crypto_data_connection_string, echo=True)
'COINBASE_100' #CoinList - symbols
'ETF_LIST' #Sample ETF
'CRYPTO_PX_HISTORY' #Crypto PX History

# If need to repopulate database sqllite, please set refresh = True
refresh = False

In [2]:
# Download all price history for Coin List 100 into CRYPTO_PX_HISTORY
# If need to rerun, please change refresh = True
if refresh == True:
    coinData.create_coinlist100
    coinData.create_sampleETF
    coinData.drop_table('CRYPTO_PX_HISTORY')
    start_date = datetime(2015, 1, 1)
    end_date = datetime.today()
    coinData.download_px_data_from_COINBASE_100(start_date, end_date)

In [3]:
# Pull Price by Name        
px_history_df = coinData.get_px_history('ETH')
display(px_history_df.head())
display(px_history_df.tail())

,symbol,open,high,low,close,adjClose,volume
date,,,,,,,
2017-11-09,ETH,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
2017-11-10,ETH,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2017-11-11,ETH,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
2017-11-12,ETH,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
2017-11-13,ETH,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984


,symbol,open,high,low,close,adjClose,volume
date,,,,,,,
2022-02-18,ETH,2884.340576,2937.308594,2761.643311,2785.727539,2785.727539,15748173433
2022-02-19,ETH,2784.872803,2826.347900,2707.378418,2763.701172,2763.701172,9774183169
2022-02-20,ETH,2763.756592,2763.756592,2585.946289,2628.648438,2628.648438,11641437834
2022-02-21,ETH,2627.665527,2752.458496,2568.254395,2573.816162,2573.816162,18646392740
2022-02-22,ETH,2577.534668,2610.277344,2518.010742,2523.676514,2523.676514,19448776704


In [6]:
# Get Price History Marix by Period - Sharpe and 30d roll beta, annual return
t_date = datetime(2022, 2, 21)
px_matrix = coinAnalytic.get_crypto_hist_martix_summary(t_date)
display(px_matrix.dropna().head())

,D0,D7_W1,M1,M3,M6,Y0_YTD,Y1,Y2,Y3,Start Date,...,X_End Close,X_Return,Y_End Date,Y_End Close,Y_Return,XY_Return,Annualized_std_dev,Average_annual_return,Sharpe_ratio,BTC_30d_roll_beta
symbol,,,,,,,,,,,,,,,,,,,,,
ADA,0.834017,1.050267,1.123233,1.839763,2.435435,1.376975,1.100520,0.058640,0.044969,2021-02-21,...,2.435435,1.212986,2022-02-21,0.834017,-0.242161,-0.657549,0.952441,0.006994,0.007343,1.032419
BAT,0.637066,0.805942,0.804666,1.074493,0.779939,1.255256,0.605740,0.266418,0.136091,2021-02-21,...,0.779939,0.287580,2022-02-21,0.637066,0.051715,-0.183185,1.195339,-0.000958,-0.000801,1.262739
BCH,279.661011,331.334412,315.208649,575.927185,676.063416,444.456177,707.592407,379.397186,142.928162,2021-02-21,...,676.063416,-0.044558,2022-02-21,279.661011,-0.604771,-0.586339,1.043153,-0.376195,-0.360633,1.113651
BNB,354.963348,403.660797,383.073914,585.294250,448.920258,527.352722,293.923584,22.184877,10.492892,2021-02-21,...,448.920258,0.527337,2022-02-21,354.963348,0.207672,-0.209295,0.893137,0.069209,0.077490,1.014486
BSV,79.982811,94.107018,96.508614,160.605362,167.910538,122.741875,242.594482,289.443390,65.390366,2021-02-21,...,167.910538,-0.307855,2022-02-21,79.982811,-0.670302,-0.523658,0.998313,-0.698957,-0.700137,0.948467


In [7]:
#Get SMA, HIGH, LOW, RS RATING
t_date = datetime(2022, 2, 21)
px_strat = coinAnalytic.get_crypto_px_strat(t_date)
display(px_strat)


,CUR_PX,SMA_50,SMA_150,SMA_200,SMA_200_20,HIGH_52WK,LOW_52WK
BTC,37075.28125,40955.09,50256.08,49405.44,48935.86,47686.81,35030.25


,CUR_PX,SMA_50,SMA_150,SMA_200,SMA_200_20,HIGH_52WK,LOW_52WK,RS_RATING
symbol,,,,,,,,
1INCH,1.367686,1.90,2.89,2.94,2.98,2.56,1.37,92.45
AAVE,128.978653,182.09,240.77,273.16,285.93,267.05,128.98,90.57
ADA,0.858236,1.15,1.59,1.79,1.81,1.59,0.86,58.49
AMP,0.025778,0.03,0.05,0.05,0.05,0.05,0.03,50.94
AR,23.921633,39.65,50.70,47.94,45.69,66.06,23.92,16.98
...,...,...,...,...,...,...,...,...
XMR,147.969238,177.81,220.26,233.74,238.56,250.21,144.48,70.75
XRP,0.703466,0.73,0.92,0.97,0.95,0.88,0.60,34.91
XTZ,2.975467,3.85,5.10,5.06,4.95,5.00,2.91,68.87


In [23]:

#file_path = Path('./Reference/sampleETF.csv')
#coinData.create_index_from_csv(file_path, 'ETF_LIST')
#coinData.create_sampleETF
sql_query = """
SELECT * FROM ETF_LIST"""
etf_list= pd.read_sql_query(sql_query, crypto_data_connection_string)
etf_summary = etf_list.merge(px_strat, left_on='symbol', right_on='symbol').groupby(['ETF','symbol']).mean().sort_values(by=['ETF','rank'])
etf_summary 

2022-02-22 00:30:16,215 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ETF_LIST")
2022-02-22 00:30:16,216 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-22 00:30:16,221 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ETF_LIST")
2022-02-22 00:30:16,222 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-22 00:30:16,226 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-02-22 00:30:16,227 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-22 00:30:16,230 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ETF_LIST")
2022-02-22 00:30:16,231 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-22 00:30:16,234 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-22 00:30:16,235 INFO sqlalchemy.engine.Engine [raw sql] ('ETF_LIST',)
2022-02-22 00:30:16,237 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("ETF_

rank        CUR_PX    SMA_50   SMA_150   SMA_200  SMA_200_20  \
ETF      symbol                                                                 
Farmdex  CAKE       1      6.892146      8.96     13.80     15.78       16.42   
         AAVE       2    128.978653    182.09    240.77    273.16      285.93   
         CRV        3      2.343160      3.75      3.89      3.49        3.35   
         RLY        5      0.233319      0.26      0.43      0.48        0.50   
         SNX        6      3.736229      5.07      7.24      8.40        8.79   
         SUSHI      7      3.280091      5.25      7.91      8.95        9.31   
         RGT        8     16.485992     22.65     28.17     24.68       23.49   
         REEF      10      0.009390      0.01      0.02      0.02        0.02   
Metadex  MANA       1      2.476375      2.82      2.66      2.21        1.98   
         SAND       2      2.900877      4.18      3.65      2.93        2.60   
         AXS        3     47.403786     63.19    100.65     92.37       90.25   
         THETA      4      2.670590      3.57      5.16      5.57        5.77   
         ENJ        5      1.387998      1.99      2.35      2.20        2.15   
         WEMIX      6      4.736940      6.12      7.50      5.86        5.32   
         WAXP       7      0.267644      0.36      0.43      0.39        0.37   
         RNDR       8      2.369028      3.21      3.23      2.72        2.50   
         SUSHI      9      3.280091      5.25      7.91      8.95        9.31   
         ONT       10      0.480171      0.56      0.78      0.84        0.86   
         UOS       11      1.126803      1.17      1.19      1.08        1.02   
         PLA       15      0.000023      0.00      0.00      0.00        0.00   
         CEEK      16      0.319291      0.42      0.41      0.31        0.28   
         CHR       17      0.377467      0.63      0.65      0.58        0.56   
Ventidex BTC        1  37075.281250  40955.09  50256.08  49405.44    48935.86   
         ETH        2   2573.816162   2969.79   3669.77   3579.08     3516.79   
         BNB        3    356.661255    421.73    491.98    475.37      466.44   
         USDC       4      0.999335      1.00      1.00      1.00        1.00   
         XRP        5      0.703466      0.73      0.92      0.97        0.95   
         ADA        6      0.858236      1.15      1.59      1.79        1.81   
         SOL        7     83.123161    116.44    165.29    151.11      144.32   
         LUNA       8      0.003708      0.01      0.01      0.01        0.01   
         AVAX       9     69.988930     81.46     85.25     74.69       67.98   

                 HIGH_52WK  LOW_52WK  RS_RATING  
ETF      symbol                                  
Farmdex  CAKE        12.33      6.89      77.83  
         AAVE       267.05    128.98      90.57  
         CRV          6.50      2.34      57.08  
         RLY          0.37      0.21      53.77  
         SNX          7.13      3.74      99.53  
         SUSHI        9.13      3.28      98.11  
         RGT         32.32     16.49      35.85  
         REEF         0.02      0.01      94.81  
Metadex  MANA         3.48      2.03       8.49  
         SAND         5.97      2.90       6.60  
         AXS         96.51     46.99       3.77  
         THETA        4.95      2.59      59.43  
         ENJ          2.86      1.39      23.58  
         WEMIX       10.34      4.17       2.83  
         WAXP         0.50      0.27      19.81  
         RNDR         5.28      2.13       5.66  
         SUSHI        9.13      3.28      98.11  
         ONT          0.70      0.45      87.74  
         UOS          1.58      0.81      16.04  
         PLA          0.00      0.00      83.96  
         CEEK         0.72      0.30       1.89  
         CHR          0.92      0.38      11.32  
Ventidex BTC      47686.81  35030.25      64.15  
         ETH       3829.56   2405.18      33.02  
         BNB        531.40    356.66      3